<a href="https://colab.research.google.com/github/berfingundem/BladderCancer/blob/main/SwinHiperparametreCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from collections import Counter
import timm
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import seaborn as sns

# --- GPU ayarı ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)

from google.colab import drive
drive.mount('/content/drive')

# --- Veri yolu ---
data_dir = "/content/drive/MyDrive/bladder_sett"  # dataset sadece HGC, LGC, NST, NTL klasörlerini içermeli
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
dataset = ImageFolder(data_dir, transform=transform)
class_names = dataset.classes  # ['HGC','LGC','NST','NTL']
num_classes = len(class_names)

# --- Parametreler ---
batch_size = 16
num_epochs = 30
k_folds = 5
patience = 5  # early stopping için sabır

# --- K-Fold Cross Validation ---
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
fold_results = {}

for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
    print(f"\n--- Fold {fold+1}/{k_folds} ---")

    # Subset oluştur
    train_subsampler = Subset(dataset, train_ids)
    val_subsampler = Subset(dataset, val_ids)

    train_loader = DataLoader(train_subsampler, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subsampler, batch_size=batch_size, shuffle=False)

    # --- Class Weights Hesapla (Fold içinde yoksa 1 olarak al) ---
    targets = [dataset.targets[i] for i in train_ids]
    class_counts = Counter(targets)
    class_weights = []

    total = float(sum(class_counts.values()))
    for i in range(num_classes):
        count = class_counts.get(i, 0)
        if count == 0:
            count = 1  # fold içinde sınıf yoksa 1 al
        class_weights.append(total / (num_classes * count))

    # CPU'da tensor oluştur, sonra CUDA'ya gönder
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

    # --- Model ---
    model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=num_classes)
    model.to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    # Early Stopping değişkenleri
    best_val_loss = np.inf
    patience_counter = 0

    train_loss_list, val_loss_list = [], []
    train_acc_list, val_acc_list = [], []

    # --- Eğitim Döngüsü ---
    for epoch in range(num_epochs):
        # --- Eğitim ---
        model.train()
        train_loss, correct, total_samples = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total_samples += labels.size(0)

        train_acc = correct / total_samples
        train_loss /= len(train_loader)

        # --- Doğrulama ---
        model.eval()
        val_loss, correct, total_samples = 0, 0, 0
        y_true, y_pred, y_score = [], [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                probs = torch.softmax(outputs, dim=1)
                _, predicted = outputs.max(1)

                correct += predicted.eq(labels).sum().item()
                total_samples += labels.size(0)

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())
                y_score.extend(probs.cpu().numpy())

        val_acc = correct / total_samples
        val_loss /= len(val_loader)

        # Kayıt
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
              f"Train Acc: {train_acc:.2f} | Val Acc: {val_acc:.2f}")

        # LR Scheduler güncelle
        scheduler.step(val_loss)

        # Early Stopping kontrol
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_wts = model.state_dict()
            best_y_true, best_y_pred, best_y_score = y_true[:], y_pred[:], y_score[:]
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping tetiklendi.")
                break

    # En iyi model yükle
    model.load_state_dict(best_model_wts)

    # Fold sonuçlarını kaydet
    fold_results[fold] = {
        'train_acc': train_acc_list,
        'val_acc': val_acc_list,
        'train_loss': train_loss_list,
        'val_loss': val_loss_list,
        'y_true': best_y_true,
        'y_pred': best_y_pred,
        'y_score': np.array(best_y_score)
    }

    # --- Konfüzyon Matrisi (sadece HGC, LGC, NST, NTL) ---
    cm = confusion_matrix(best_y_true, best_y_pred, labels=list(range(num_classes)))
    plt.figure(figsize=(6, 5), dpi=300)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Fold {fold+1} - Konfüzyon Matrisi')
    plt.xlabel('Tahmin')
    plt.ylabel('Gerçek')
    plt.show()

    # --- ROC Eğrisi ---
    y_true_bin = label_binarize(best_y_true, classes=list(range(num_classes)))
    fpr, tpr, roc_auc = {}, {}, {}
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], fold_results[fold]['y_score'][:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    plt.figure(figsize=(7, 6), dpi=300)
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], label=f"{class_names[i]} (AUC={roc_auc[i]:.2f})")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f'Fold {fold+1} - ROC Eğrisi')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()

# --- Fold ortalama doğruluk ---
all_val_acc = [max(fold_results[f]['val_acc']) for f in fold_results]
print("\nK-Fold ortalama doğruluk:", np.mean(all_val_acc))
